In [7]:
import statsmodels.api as sm
import pandas as pd

In [8]:
df_OHE = pd.read_csv('../DataSet/RegressionData/healthinsurance_OHE.csv')
df_LE = pd.read_csv('../DataSet/RegressionData/healthinsurance_LE.csv')

## Statistical Regression Analysis

### OHE

In [9]:
# use statsmodel for linear regression

X = df_OHE.drop('claim', axis=1)

y = df_OHE['claim']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  claim   R-squared:                       0.770
Model:                            OLS   Adj. R-squared:                  0.768
Method:                 Least Squares   F-statistic:                     319.1
Date:                Mon, 17 Feb 2025   Prob (F-statistic):               0.00
Time:                        12:50:19   Log-Likelihood:            -1.3761e+05
No. Observations:               13648   AIC:                         2.755e+05
Df Residuals:                   13505   BIC:                         2.766e+05
Df Model:                         142                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

### LE

In [10]:
X = df_LE.drop('claim', axis=1)

y = df_LE['claim']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  claim   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     3339.
Date:                Mon, 17 Feb 2025   Prob (F-statistic):               0.00
Time:                        12:50:19   Log-Likelihood:            -1.3829e+05
No. Observations:               13648   AIC:                         2.766e+05
Df Residuals:                   13635   BIC:                         2.767e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                5043.3053    

## 2. Part Investing/Increasing the regression performance

In [11]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from scipy.stats import loguniform
from sklearn.metrics import mean_absolute_percentage_error, r2_score, make_scorer
import statsmodels.api as sm

# Define features & target
X = df_OHE.drop('claim', axis=1)
y = df_OHE['claim']

# Add constant term for models that need it (like Ridge, Lasso)
X = sm.add_constant(X)

# First split: 85% train+val, 15% test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Second split: 70% train, 15% validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  

# Creating a pipeline for scaling and modeling
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge())  # Default model, will be replaced in search
])

# Defining a larger hyperparameter grid
param_grid = {
    "model": [Ridge(), Lasso(), ElasticNet()], 
    "model__alpha": loguniform(1e-4, 1e3),  
    "model__fit_intercept": [True, False],
}

mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)  # Minimize error
# Performing Randomized Search with 5-Fold Cross Validation
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=200,  # More iterations for thorough search
    cv=5,
    scoring=mape_scorer,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fitting the model (on training set only)
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Evaluate on validation set
val_r2 = best_model.score(X_val, y_val)

# Predictions on training & test sets
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Compute R² and MAPE for train & test sets
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
mape_train = mean_absolute_percentage_error(y_train, y_train_pred) * 100  # Convert to %
mape_test = mean_absolute_percentage_error(y_test, y_test_pred) * 100  # Convert to %

# Print results
print("Best Model:", best_model)
print("Best Cross-Validation Score (R²):", random_search.best_score_)
print(f"Validation Set R²: {val_r2:.4f}")

print(f"Train Set R²: {r2_train:.4f}")
print(f"Train Set MAPE: {mape_train:.2f}%")

print(f"Test Set R²: {r2_test:.4f}")
print(f"Test Set MAPE: {mape_test:.2f}%")



Best Model: Pipeline(steps=[('scaler', StandardScaler()),
                ('model', Lasso(alpha=46.85908054580002))])
Best Cross-Validation Score (R²): -0.3791375593085139
Validation Set R²: 0.7576
Train Set R²: 0.7682
Train Set MAPE: 37.42%
Test Set R²: 0.7819
Test Set MAPE: 35.71%


In [12]:
# Define features & target
X = df_LE.drop('claim', axis=1)
y = df_LE['claim']

# Add constant term for models that need it (like Ridge, Lasso)
X = sm.add_constant(X)

# First split: 85% train+val, 15% test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Second split: 70% train, 15% validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  

# Creating a pipeline for scaling and modeling
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge())  # Default model, will be replaced in search
])

# Defining a larger hyperparameter grid
param_grid = {
    "model": [Ridge(), Lasso(), ElasticNet()], 
    "model__alpha": loguniform(1e-4, 1e3),  
    "model__fit_intercept": [True, False],
}

mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)  # Minimize error
# Performing Randomized Search with 5-Fold Cross Validation
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=200,  # More iterations for thorough search
    cv=5,
    scoring=mape_scorer,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fitting the model (on training set only)
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Evaluate on validation set
val_r2 = best_model.score(X_val, y_val)

# Predictions on training & test sets
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Compute R² and MAPE for train & test sets
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
mape_train = mean_absolute_percentage_error(y_train, y_train_pred) * 100  # Convert to %
mape_test = mean_absolute_percentage_error(y_test, y_test_pred) * 100  # Convert to %

# Print results
print("Best Model:", best_model)
print("Best Cross-Validation Score (R²):", random_search.best_score_)
print(f"Validation Set R²: {val_r2:.4f}")

print(f"Train Set R²: {r2_train:.4f}")
print(f"Train Set MAPE: {mape_train:.2f}%")

print(f"Test Set R²: {r2_test:.4f}")
print(f"Test Set MAPE: {mape_test:.2f}%")


Best Model: Pipeline(steps=[('scaler', StandardScaler()),
                ('model', Lasso(alpha=46.85908054580002))])
Best Cross-Validation Score (R²): -0.3996223874869287
Validation Set R²: 0.7363
Train Set R²: 0.7443
Train Set MAPE: 39.89%
Test Set R²: 0.7632
Test Set MAPE: 37.97%
